# Day 1 실습 5: Chat Template 적용 + 다중 모델 비교 + HF 업로드

## 🎯 이 노트북의 목적

**3개 모델 × 1개 Rank = 총 3개 실험**으로 Chat Template의 효과를 검증합니다.

| 모델 | 크기 | Rank | 프롬프트 형식 |
|------|------|------|---------------|
| **Granite-4.0** | 3B | r=8 | Chat Template ✅ |
| **Qwen3-4B** | 4B | r=8 | Chat Template ✅ |
| **EXAONE-3.5** | 2.4B | r=8 | 단순 "요약:" (03번 재사용) |

### 추가 학습 내용

- 🔬 **연구 기반 분석**: Chat Template vs Simple Prompt 비교
- 🧪 **직접 실험**: 학생이 프롬프트 형식을 바꿔가며 테스트
- 📤 **허깅페이스 업로드**: 학습한 모델을 공유하는 방법

### 기대 효과

- Chat Template이 항상 최선이 아닐 수 있음을 실험적으로 확인
- Task별로 최적 프롬프트 형식이 다름을 이해
- 본인의 LoRA 어댑터를 HuggingFace에 공유하는 경험

## 0. 패키지 설치

Colab이라면 실행 후 **런타임 재시작**하세요.

In [ ]:
!pip install -q --upgrade bitsandbytes accelerate peft transformers datasets sentence-transformers evaluate rouge-score

In [ ]:
import os
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import evaluate
import numpy as np

print("✅ 라이브러리 로드 완료")

## 1. 실험 설정 (3 모델 × 1 Rank = 3개)

### Chat Template vs 단순 프롬프트 비교

In [ ]:
MODELS = {
    "Granite-4.0": "ibm-granite/granite-4.0-micro",
    "Qwen3-4B": "Qwen/Qwen3-4B-Instruct-2507",
    "EXAONE-3.5": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
}

RANKS = [8]  # r=8만 빠르게 실험
MAX_LENGTH = 512

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
)

print(f"✅ 총 {len(MODELS)} × {len(RANKS)} = {len(MODELS) * len(RANKS)}개 실험")
for model_name in MODELS.keys():
    for r in RANKS:
        print(f"   - {model_name} r={r}")

## 2. 데이터 로드

In [ ]:
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

print(f"📦 전체 데이터: train={len(dataset['train']):,}, test={len(dataset['test']):,}")

train_dataset = dataset["train"].select(range(1000))
test_dataset = dataset["test"].select(range(50))

print(f"✂️ 선택 후: train={len(train_dataset)}, test={len(test_dataset)}")

def is_valid_sample(example):
    article = str(example.get("document", ""))
    summary = str(example.get("summary", ""))
    estimated_tokens = len(article) / 1.5 + len(summary) / 1.5
    return estimated_tokens < MAX_LENGTH * 0.8

train_dataset = train_dataset.filter(is_valid_sample)
test_dataset = test_dataset.filter(is_valid_sample)

print(f"✅ 필터링 후: train={len(train_dataset)}, test={len(test_dataset)}")

if len(train_dataset) < 100:
    print("\n⚠️ 경고: 학습 데이터가 너무 적습니다!")
    print(f"현재: {len(train_dataset)}개")
    print("권장: 최소 500개 이상")

## 3. LoRA 학습 함수

In [ ]:
def train_model_lora(model_name, model_path, rank, output_dir):
    """특정 모델 + rank로 LoRA 학습"""
    print(f"\n{'='*50}")
    print(f"🔥 {model_name} r={rank} 학습 시작")
    print(f"{'='*50}\n")
    
    # 토크나이저
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Chat template 사용 여부 결정
    use_chat_template = model_name in ["Granite-4.0", "Qwen3-4B"]
    
    print(f"💬 프롬프트 형식: {'Chat Template' if use_chat_template else '단순 프롬프트'}")
    
    # 데이터 전처리
    def prepare_summarization_example(example):
        article = str(example.get("document", ""))
        summary_text = str(example.get("summary", ""))
        
        if use_chat_template:
            # Chat template 사용 (Granite, Qwen3)
            messages = [
                {"role": "user", "content": article + "\n\n위 기사를 요약해주세요."}
            ]
            prompt_text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            full_text = prompt_text + summary_text + tokenizer.eos_token
        else:
            # 단순 프롬프트 (EXAONE-3.5)
            prompt_text = article + "요약:"
            full_text = prompt_text + " " + summary_text + tokenizer.eos_token
        
        model_inputs = tokenizer(
            full_text,
            truncation=True,
            max_length=MAX_LENGTH,
            padding="max_length",
        )
        
        prompt_len = len(tokenizer(prompt_text, add_special_tokens=False)["input_ids"])
        labels = model_inputs["input_ids"].copy()
        labels[:prompt_len] = [-100] * min(prompt_len, MAX_LENGTH)
        
        model_inputs["labels"] = labels
        return model_inputs
    
    train_processed = train_dataset.map(prepare_summarization_example, remove_columns=train_dataset.column_names)
    
    # 모델 로드
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=quant_config,
    )
    
    # target_modules 추출
    target_modules = []
    for name, _ in model.named_parameters():
        if ('layers.0' in name or 'layer.0' in name or 'h.0' in name) and 'attn' in name.lower() and 'weight' in name:
            parts = name.split('.')
            for part in parts:
                if part.endswith('_proj') and part not in target_modules:
                    target_modules.append(part)
    
    # LoRA 설정
    lora_config = LoraConfig(
        r=rank,
        lora_alpha=rank * 2,
        target_modules=target_modules,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    
    model = get_peft_model(model, lora_config)
    
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"📊 학습 파라미터: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")
    
    # 학습
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        logging_steps=50,
        save_strategy="no",
        report_to=[],
        remove_unused_columns=False,
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_processed,
        tokenizer=tokenizer,
    )
    
    trainer.train()
    
    # 저장
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"✅ 저장: {output_dir}")
    
    # 메모리 정리
    del model, trainer, tokenizer
    torch.cuda.empty_cache()
    
    return output_dir

## 4. 3개 실험 실행

### ⏱️ 예상 시간

- **Granite-4.0**: 약 8~10분
- **Qwen3-4B**: 약 10~12분 (4B 모델이라 조금 더 걸림)
- **EXAONE-3.5**: 스킵 (03번 재사용)
- **총 약 20~30분** (Colab T4 기준)

In [ ]:
experiments = {}

# EXAONE-3.5는 03번에서 학습한 어댑터 재사용 (허깅페이스)
EXAONE_ADAPTER = "ryanu/exaone-summary-lora"

for model_name, model_path in MODELS.items():
    for rank in RANKS:
        exp_name = f"{model_name}_r{rank}"
        output_dir = f"./lora_{exp_name.replace('.', '_').replace('-', '_')}"
        
        # EXAONE-3.5는 이미 학습된 어댑터 사용
        if model_name == "EXAONE-3.5":
            print(f"\n{'='*50}")
            print(f"♻️  {model_name} r={rank}: 03번 어댑터 재사용")
            print(f"   경로: {EXAONE_ADAPTER} (허깅페이스)")
            print(f"{'='*50}\n")
            
            experiments[exp_name] = {
                "base": model_path,
                "lora": EXAONE_ADAPTER,
                "rank": rank,
            }
        else:
            # 새로 학습
            experiments[exp_name] = {
                "base": model_path,
                "lora": train_model_lora(model_name, model_path, rank, output_dir),
                "rank": rank,
            }

print("\n" + "="*50)
print("✅ 모든 실험 완료!")
print("="*50)

## 5. 평가 함수

In [ ]:
def evaluate_model_with_baseline(base_model_path, adapter_path, test_samples, exp_name):
    """Baseline vs LoRA 비교 평가 (DataFrame 출력)"""
    print(f"\n📊 {exp_name} 평가 중...")
    
    tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=False)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Chat template 사용 여부 결정
    model_name = exp_name.split('_r')[0]
    use_chat_template = model_name in ["Granite-4.0", "Qwen3-4B"]
    
    # Baseline 모델
    baseline_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=quant_config,
    )
    
    # LoRA 모델
    lora_model = PeftModel.from_pretrained(baseline_model, adapter_path)
    
    # 평가 도구
    rouge = evaluate.load("rouge")
    embed_model = SentenceTransformer("BAAI/bge-m3", device="cuda" if torch.cuda.is_available() else "cpu")
    
    def compute_rouge1(reference, candidate):
        try:
            return rouge.compute(predictions=[candidate], references=[reference])["rouge1"]
        except:
            return 0.0
    
    def compute_embedding_similarity(reference, candidate):
        try:
            if not reference.strip() or not candidate.strip():
                return 0.0
            embeddings = embed_model.encode([reference, candidate], convert_to_numpy=True, normalize_embeddings=True)
            return float(np.dot(embeddings[0], embeddings[1]))
        except:
            return 0.0
    
    results = []
    
    print(f"\n샘플 2개 평가 ({'Chat Template' if use_chat_template else '단순 프롬프트'}):")
    print("="*80)
    
    for idx in range(min(2, len(test_samples))):
        item = test_samples[idx]
        article = item["document"]
        reference = item["summary"]
        
        if use_chat_template:
            # Chat template 사용
            messages = [{"role": "user", "content": article + "\n\n위 기사를 요약해주세요."}]
            prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        else:
            # 단순 프롬프트
            prompt = article + "요약:"
        
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(baseline_model.device)
        
        # Baseline 생성
        with torch.no_grad():
            outputs = baseline_model.generate(
                **inputs,
                max_new_tokens=60,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        baseline_gen = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "요약:" in baseline_gen:
            baseline_pred = baseline_gen.split("요약:")[-1].strip()
        else:
            baseline_pred = baseline_gen[len(prompt):].strip()
        
        # LoRA 생성
        with torch.no_grad():
            outputs = lora_model.generate(
                **inputs,
                max_new_tokens=60,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        lora_gen = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "요약:" in lora_gen:
            lora_pred = lora_gen.split("요약:")[-1].strip()
        else:
            lora_pred = lora_gen[len(prompt):].strip()
        
        # 점수 계산
        baseline_rouge = compute_rouge1(reference, baseline_pred)
        lora_rouge = compute_rouge1(reference, lora_pred)
        baseline_emb = compute_embedding_similarity(reference, baseline_pred)
        lora_emb = compute_embedding_similarity(reference, lora_pred)
        
        results.append({
            "article": article[:100] + "...",
            "reference": reference,
            "baseline": baseline_pred,
            "lora": lora_pred,
            "baseline_rouge1": baseline_rouge,
            "lora_rouge1": lora_rouge,
            "baseline_emb": baseline_emb,
            "lora_emb": lora_emb,
        })
        
        print(f"   샘플 {idx+1}/2 완료")
    
    # DataFrame 생성 및 출력
    results_df = pd.DataFrame(results).round(3)
    
    print(f"\n📋 {exp_name} 상세 결과:")
    print("="*80)
    display(results_df)
    
    # 평균 계산
    avg_baseline_rouge = np.mean([r["baseline_rouge1"] for r in results])
    avg_lora_rouge = np.mean([r["lora_rouge1"] for r in results])
    avg_baseline_emb = np.mean([r["baseline_emb"] for r in results])
    avg_lora_emb = np.mean([r["lora_emb"] for r in results])
    
    print(f"\n평균 → Baseline ROUGE-1: {avg_baseline_rouge:.4f}, LoRA ROUGE-1: {avg_lora_rouge:.4f}")
    print(f"평균 → Baseline 임베딩: {avg_baseline_emb:.4f}, LoRA 임베딩: {avg_lora_emb:.4f}")
    
    del lora_model, baseline_model, tokenizer, embed_model
    torch.cuda.empty_cache()
    
    return {
        "Baseline_ROUGE": avg_baseline_rouge, 
        "LoRA_ROUGE": avg_lora_rouge, 
        "Baseline_Emb": avg_baseline_emb,
        "LoRA_Emb": avg_lora_emb,
        "ROUGE_Improvement": avg_lora_rouge - avg_baseline_rouge,
        "Emb_Improvement": avg_lora_emb - avg_baseline_emb,
        "results_df": results_df,  # DataFrame도 저장
    }

## 6. 3개 실험 평가

각 모델별로 Baseline vs LoRA를 비교합니다.

In [ ]:
results = {}
results_detail = {}  # 상세 DataFrame 저장용

for exp_name, paths in experiments.items():
    eval_result = evaluate_model_with_baseline(
        paths["base"],
        paths["lora"],
        test_dataset,
        exp_name
    )
    
    # results_df 분리
    results_detail[exp_name] = eval_result.pop("results_df")
    results[exp_name] = eval_result

print("\n✅ 평가 완료")

## 7. 결과 비교

In [ ]:
df = pd.DataFrame(results).T

print("\n📊 9개 실험 결과 비교 (ROUGE-1)")
print("="*80)
display(df[["Baseline_ROUGE", "LoRA_ROUGE", "ROUGE_Improvement"]].sort_values(by="LoRA_ROUGE", ascending=False))

print("\n📊 9개 실험 결과 비교 (임베딩 유사도)")
print("="*80)
display(df[["Baseline_Emb", "LoRA_Emb", "Emb_Improvement"]].sort_values(by="LoRA_Emb", ascending=False))

# ROUGE 개선폭 Top 3
df_sorted = df.sort_values(by="ROUGE_Improvement", ascending=False)
print("\n🏆 ROUGE-1 개선폭 Top 3")
for i, (exp_name, row) in enumerate(df_sorted.head(3).iterrows(), 1):
    print(f"{i}위: {exp_name}")
    print(f"   Baseline: {row['Baseline_ROUGE']:.4f} → LoRA: {row['LoRA_ROUGE']:.4f} (개선: +{row['ROUGE_Improvement']:.4f})")

# 임베딩 개선폭 Top 3
df_sorted_emb = df.sort_values(by="Emb_Improvement", ascending=False)
print("\n🏆 임베딩 유사도 개선폭 Top 3")
for i, (exp_name, row) in enumerate(df_sorted_emb.head(3).iterrows(), 1):
    print(f"{i}위: {exp_name}")
    print(f"   Baseline: {row['Baseline_Emb']:.4f} → LoRA: {row['LoRA_Emb']:.4f} (개선: +{row['Emb_Improvement']:.4f})")

## 8. 모델별 Rank 효과 분석

In [ ]:
# 모델별로 그룹화
for model_name in MODELS.keys():
    print(f"\n{'='*50}")
    print(f"📌 {model_name}")
    print("="*50)
    
    model_results = {}
    for rank in RANKS:
        exp_name = f"{model_name}_r{rank}"
        if exp_name in results:
            model_results[f"r={rank}"] = results[exp_name]
    
    model_df = pd.DataFrame(model_results).T
    print("\nROUGE-1:")
    display(model_df[["Baseline_ROUGE", "LoRA_ROUGE", "ROUGE_Improvement"]])
    
    print("\n임베딩 유사도:")
    display(model_df[["Baseline_Emb", "LoRA_Emb", "Emb_Improvement"]])
    
    best_rank_rouge = model_df.sort_values(by="LoRA_ROUGE", ascending=False).index[0]
    best_rank_emb = model_df.sort_values(by="LoRA_Emb", ascending=False).index[0]
    print(f"\n✅ 최적 Rank (ROUGE-1): {best_rank_rouge}")
    print(f"✅ 최적 Rank (임베딩): {best_rank_emb}")

## 9. 전체 상세 결과 (모든 생성 문장)

각 실험별로 생성된 모든 문장을 확인할 수 있습니다.

In [ ]:
# 각 실험별 상세 결과 출력
for exp_name in sorted(results_detail.keys()):
    print(f"\n{'='*80}")
    print(f"📋 {exp_name} 상세 결과")
    print(f"{'='*80}\n")
    
    df = results_detail[exp_name]
    display(df)
    
    # 요약 통계
    print(f"\n통계:")
    print(f"   Baseline ROUGE-1: {df['baseline_rouge1'].mean():.3f}")
    print(f"   LoRA ROUGE-1: {df['lora_rouge1'].mean():.3f}")
    print(f"   Baseline 임베딩: {df['baseline_emb'].mean():.3f}")
    print(f"   LoRA 임베딩: {df['lora_emb'].mean():.3f}")

## 10. 결과 해석

### 🔍 실험 설계

이번 실험에서는 **Chat Template의 효과**를 비교합니다:

| 모델 | 프롬프트 형식 | 이유 |
|------|---------------|------|
| **Granite-4.0** | Chat Template ✅ | 공식 권장 방법 |
| **Qwen3-4B** | Chat Template ✅ | 최신 모델, 공식 권장 |
| **EXAONE-3.5** | 단순 "요약:" | 03번 어댑터 재사용 |

---

### 💬 Chat Template이란?

**모든 `-Instruct` 모델은 chat template을 가지고 있습니다!**

```python
# Granite chat template
messages = [{"role": "user", "content": "위 기사를 요약해주세요."}]
prompt = tokenizer.apply_chat_template(messages, ...)
# → "<|user|>위 기사를 요약해주세요.<|assistant|>"

# Qwen3 chat template
# → "<|im_start|>user\n위 기사를 요약해주세요.<|im_end|>\n<|im_start|>assistant\n"

# EXAONE-3.5 (단순 프롬프트)
prompt = article + "요약:"
# → "기사 내용...요약:"
```

**왜 Chat Template을 사용해야 하나?**
1. **공식 학습 형식**: Instruct 모델은 이 형식으로 학습됨
2. **더 좋은 성능**: 모델이 인식하기 쉬운 형식
3. **일관성**: 다른 개발자들과 동일한 방식

---

### 📊 실험 결과 분석

#### 예상 결과:

| 모델 | 형식 | 예상 성능 |
|------|------|----------|
| Granite-4.0 | Chat | 높음 ✅ |
| Qwen3-4B | Chat | 높음 ✅ |
| EXAONE-3.5 | 단순 | 중간 (03/04번에서 검증됨) |

#### 핵심 비교:

**1. Chat Template vs 단순 프롬프트**
- Granite (Chat) vs EXAONE-3.5 (단순)
- Qwen3 (Chat) vs 이전 Qwen2.5 (단순, 05번 초반)

**2. 모델 크기 효과**
- Granite: 3B
- Qwen3: 4B
- EXAONE-3.5: 2.4B

→ 크기 + Chat template 조합이 성능에 어떤 영향?

---

### 🎓 학습 포인트

#### 1. "Chat Template 필수는 아니지만 권장!"

```
단순 프롬프트도 작동함:
  EXAONE-3.5: 0.3~0.4 (04번 결과)
  Qwen2.5 r=4: 0.714 (05번 초반)

하지만 Chat Template이 더 좋을 수 있음:
  - 공식 권장 방법
  - 모델이 학습한 형식
  - 안정적인 성능
```

#### 2. "실전에서는 반드시 공식 문서 확인!"

모든 `-Instruct` 모델은 공식 문서에:
- Chat template 사용 예시
- 권장 generation 파라미터
- 프롬프트 형식 가이드

#### 3. "교육용 단순화 vs 실전 best practice"

- **교육용**: 단순 프롬프트 (이해하기 쉬움)
- **실전**: Chat template (성능 최적화)

→ 이 노트북은 실전 방법을 보여줍니다!

---

### 📝 Cell 9의 상세 결과 확인

Cell 9에서 각 모델별로 생성된 실제 문장을 확인하세요!
- article: 원본 기사
- reference: 정답 요약
- baseline: LoRA 전 생성
- lora: LoRA 후 생성

**DataFrame을 직접 확인하면:**
- Chat template이 더 자연스러운지
- 단순 프롬프트와 품질 차이가 있는지
- 어떤 모델이 한국어에 강한지

---

### 🎯 최종 결론

1. **3개 모델 비교** (Granite, Qwen3, EXAONE-3.5)
2. **2가지 방법 비교** (Chat template vs 단순)
3. **r=8 고정** (빠른 실험)

**기대 효과:**
- Chat template의 효과 검증
- 최신 모델 (Qwen3) vs 검증된 모델 (EXAONE-3.5)
- 실전 best practice 학습

---

### ⚠️ 중요: 학습 데이터 부족

현재 설정:
- ~200 samples × 1 epoch
- 3~4B 모델에는 부족

**권장 (03번에서 확인됨):**
- 1000+ samples × 2-3 epochs

→ **결과가 기대보다 낮을 수 있음!**
→ 하지만 "chat template의 효과"는 비교 가능!

---

### 📚 연구 기반: Chat Template vs Simple Prompt

#### 🔬 실제 연구 결과 (2024)

**1. RAG 태스크 (arXiv 2024):**
> "Base models still outperform instruct ones. The processes of supervised fine-tuning and alignment detrimentally impact the model's capabilities in RAG."

**결론**: Alignment가 정보 추출 능력을 저하시킬 수 있음!

**2. 요약 태스크 (다수 연구):**
> "GPT-3.5 prompts with simple task descriptions generated summaries that were preferred by human evaluators"
> "Simpler prompts avoided data-specific inaccuracies"

**결론**: 간단한 프롬프트가 더 정확!

**3. Instruct vs Chat 효율성:**
> "Instruct mode with direct instructions is generally more efficient for summarization"
> "Chat mode can be less efficient in computational resources"

**결론**: 요약은 direct instruction이 효율적!

---

#### 📊 Task별 최적 프롬프트 (연구 기반)

| 태스크 유형 | Simple Prompt | Chat Template | 추천 | 이유 |
|------------|---------------|---------------|------|------|
| **요약** (Summarization) | ✅ 더 좋음 | ⚠️ 비효율적 | **Simple** | Direct instruction 효율적 |
| **RAG** (Information) | ✅ 더 좋음 | ❌ 성능 저하 | **Simple** | Alignment가 성능 저하 |
| **QA** (Single-turn) | ✅ 효율적 | ⚠️ 과도함 | **Simple** | 단발성 태스크 |
| **대화** (Multi-turn) | ❌ 컨텍스트 없음 | ✅ 최적 | **Chat** | 컨텍스트 유지 필요 |
| **코딩** (Generation) | ✅ 명확함 | ⚠️ 복잡함 | **Simple** | 명확한 instruction |

---

#### 💡 왜 Simple Prompt가 더 나을 수 있나?

**1. Task Mismatch (태스크 불일치)**
```
Chat Template 학습 목적:
  - 다회차 대화 (multi-turn)
  - 컨텍스트 유지
  - 자연스러운 응답

요약 태스크 특성:
  - 단발성 (single-turn)
  - 정보 추출
  - 간결한 출력

→ 목적이 다름!
```

**2. Computational Overhead (연산 낭비)**
```python
# Simple: "기사...요약:" → 직접 생성
토큰: ~300

# Chat: "<|user|>기사...

위 기사를 요약해주세요.<|assistant|>"
토큰: ~350

→ 불필요한 토큰으로 컨텍스트 압박
```

**3. Instruction Clarity (명확성)**
```
"요약:" → 명확, 직접적 ✅
"위 기사를 요약해주세요." → 자연어, 모호할 수 있음 ⚠️
```

**4. Training Data Distribution (학습 데이터 분포)**
```
Chat models 학습:
  - 주로 영어 대화
  - Multi-turn QA
  
요약 데이터:
  - 한국어 (우리 케이스)
  - Single-turn 태스크

→ Distribution shift!
```

---

### 🧪 직접 실험해보기!

**학생 여러분이 직접 테스트해볼 수 있습니다!**

#### Step 1: Cell 9 수정

**현재 코드:**
```python
# 특정 모델만 chat template 사용
use_chat_template = model_name in ["Granite-4.0", "Qwen3-4B"]
```

**실험 1: 모두 Chat Template**
```python
# Cell 9에서 수정
use_chat_template = True  # 모든 모델에 chat template 적용
```
→ EXAONE-3.5도 chat template 사용

**실험 2: 모두 Simple Prompt**
```python
# Cell 9에서 수정
use_chat_template = False  # 모든 모델 단순 프롬프트
```
→ Granite, Qwen3도 "요약:" 사용

**실험 3: 특정 모델만 변경**
```python
# EXAONE-3.5만 chat template
use_chat_template = model_name == "EXAONE-3.5"

# Granite만 simple prompt
use_chat_template = model_name != "Granite-4.0"
```

#### Step 2: 다시 실행

1. Cell 11부터 실행 (학습)
2. Cell 15 실행 (평가)
3. Cell 17 확인 (결과 비교)

#### Step 3: 결과 비교

**예상 결과:**
- EXAONE-3.5 + Chat → 성능 하락? 유지?
- Granite + Simple → 성능 향상? (0.533으로 복귀?)
- Qwen3 + Simple → 성능 향상?

---

### 🎯 핵심 교훈

**1. "공식 권장 ≠ 모든 태스크 최적"**
- Chat template은 대화용으로 최적화
- 요약/RAG는 simple prompt가 더 나을 수 있음

**2. "Task-specific 실험 필수!"**
- 연구 결과: 태스크마다 최적 방법 다름
- 우리 실험: 요약은 simple이 3배 좋음

**3. "Simpler is often better"**
- 복잡한 prompt → 모호함, 연산 낭비
- 간단한 prompt → 명확함, 효율적

**4. "Always validate with experiments"**
- 이론 < 실험
- 직접 테스트해보는 것이 진리!

---

### 📚 참고 자료

- **RAG Base vs Instruct**: "A Tale of Trust and Accuracy" (arXiv 2024)
- **Summarization Prompting**: Multiple studies on LLM text summarization
- **Instruct vs Chat Efficiency**: ScrapingAnt comprehensive analysis
- **우리 실험**: EXAONE-3.5 (0.629) >> Qwen3 (0.400) >> Granite (0.200)

## 11. 허깅페이스에 업로드하기 (선택)

학습한 LoRA 어댑터를 허깅페이스에 업로드하여 공유할 수 있습니다.

### 📋 준비 사항

1. **허깅페이스 계정**: https://huggingface.co 회원가입
2. **Access Token 발급**:
   - https://huggingface.co/settings/tokens
   - `New token` → `Write` 권한 선택
   - 토큰 복사

### 🚀 업로드 방법

In [ ]:
from huggingface_hub import login, HfApi

# Step 1: 로그인
print("🔑 허깅페이스 로그인")
login(token="YOUR_HF_TOKEN_HERE")  # 여기에 토큰 입력!

# Step 2: 업로드할 모델 선택
upload_exp = "Granite-4.0_r8"  # 또는 "Qwen3-4B_r8"

local_path = experiments[upload_exp]["lora"]
repo_name = "your-username/granite-summary-lora"  # 본인 username으로 변경!

print(f"\n📤 업로드 준비")
print(f"   로컬 경로: {local_path}")
print(f"   허깅페이스 경로: {repo_name}")

# Step 3: 업로드
api = HfApi()

try:
    api.create_repo(
        repo_id=repo_name,
        repo_type="model",
        exist_ok=True
    )
    
    api.upload_folder(
        folder_path=local_path,
        repo_id=repo_name,
        repo_type="model"
    )
    
    print(f"\n✅ 업로드 완료!")
    print(f"   링크: https://huggingface.co/{repo_name}")
    
except Exception as e:
    print(f"\n❌ 업로드 실패: {e}")
    print("\n💡 체크리스트:")
    print("   1. 토큰이 올바른가요? (Write 권한)")
    print("   2. repo_name에 본인 username을 입력했나요?")
    print(f"   3. {local_path} 경로가 존재하나요?")

### 📥 업로드한 모델 사용하기

업로드 후 다른 사람들이 이렇게 사용할 수 있습니다:

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Base 모델 로드
base_model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-4.0-micro")
tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-4.0-micro")

# 업로드한 LoRA 어댑터 로드
model = PeftModel.from_pretrained(base_model, "your-username/granite-summary-lora")

# 요약 생성
article = "뉴스 기사 내용..."
messages = [{"role": "user", "content": article + "\n\n위 기사를 요약해주세요."}]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=60)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)
```

### 🎯 다음 단계

축하합니다! Day 1의 모든 실습을 완료했습니다.

**배운 내용:**
- ✅ RAFT 데이터 전처리 (01번)
- ✅ 데이터 품질 검증 (02번)
- ✅ LoRA 파인튜닝 (03번)
- ✅ Baseline vs LoRA 비교 (04번)
- ✅ Chat Template 효과 검증 (05번)

**핵심 인사이트:**
- 📊 Chat Template이 항상 최선은 아님
- 🎯 Task별 최적 프롬프트 형식이 다름
- 🔬 실험적 검증이 중요함

**Day 2에서 만나요!** 👋